<a href="https://colab.research.google.com/github/gmrammohan15/EVA4/blob/master/S8-Assignment/S8_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [140]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [141]:
import sys
sys.path.append('/content/gdrive/My Drive/S8-Assignment/')
!ls /content/gdrive/My\ Drive/S8-Assignment

dataloader.py	    __init__.py       prepare_dataset.py  stats.py
data_transforms.py  misclassified.py  README.md		  transforms.py
eval.py		    models	      resnet.py


In [0]:
import numpy as np
import data_transforms
from torchvision import transforms

import importlib
importlib.reload(data_transforms)

transformer = data_transforms.Transforms(normalize=True, mean=(0.4914, 0.4822, 0.4465), stdev=(0.247, 0.243, 0.261))

#Train transformations
train_transforms = transformer.train_transforms([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32,padding=4),
    #transforms.RandomErasing(value = tuple((0.4914, 0.4822, 0.4465)))
])

# Test  transformations
test_transforms = transformer.test_transforms()

In [151]:
from torchvision import datasets
train_data = datasets.CIFAR10('./data', train=True, download=True, transform=train_transforms)
test_data = datasets.CIFAR10('./data', train=False, download=True, transform=test_transforms)

Files already downloaded and verified
Files already downloaded and verified


In [0]:
import dataloader

import importlib
importlib.reload(dataloader)

dl = dataloader.DataLoader(batch_size=128)

# train dataloader
train_loader = dl.load(train_data)

# test dataloader
test_loader = dl.load(test_data)

In [0]:
import matplotlib.pyplot as plt
import torchvision
import numpy as np

# functions to show an image
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

# get some random training images
dataiter = iter(train_loader)
images, labels  = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(32)))

In [153]:
#from models import resnet
import resnet

import torch

import importlib
importlib.reload(resnet)
#importlib.reload(eval)


model = resnet.ResNet18()
use_cuda = torch.cuda.is_available()
model.device = torch.device("cuda" if use_cuda else "cpu")
model.to(model.device)
model.summary((3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
        BasicBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
       BasicBlock-12           [-1, 64, 32, 32]               0
           Conv2d-13          [-1, 128, 16, 16]          73,728
      BatchNorm2d-14          [-1, 128,

In [0]:
import torch.optim as optim
import torch.nn as nn
from torch.optim.lr_scheduler import OneCycleLR
from torch.optim.lr_scheduler import StepLR


EPOCHS = 10
l2_loss = 0.0001
l1_lambda = 0
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=l2_loss)

#scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
schedular = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.5, steps_per_epoch=len(train_loader), epochs=EPOCHS)


model.evaluate(optimizer, train_loader, test_loader, EPOCHS, criterion, schedular, True, l1_lambda)

Epoch 0, Train Accuracy: 0.4166400134563446 , Test Accuracy: 0.48579999804496765
Epoch 1, Train Accuracy: 0.6097400188446045 , Test Accuracy: 0.6317999958992004
Epoch 2, Train Accuracy: 0.6985800266265869 , Test Accuracy: 0.6590999960899353
Epoch 3, Train Accuracy: 0.7588000297546387 , Test Accuracy: 0.7731999754905701
Epoch 4, Train Accuracy: 0.7932800054550171 , Test Accuracy: 0.7785999774932861
Epoch 5, Train Accuracy: 0.8169800043106079 , Test Accuracy: 0.8064000010490417
Epoch 6, Train Accuracy: 0.8353800177574158 , Test Accuracy: 0.8190000057220459
